# Enrichment test
What is the probability to randomly select at least k "changed" reactions out of n "changed" reactions when selecting N out of M reactions.   
* k: number of diferentially expressed reactions in a subsystem,
* n: number of diferentially expressed reactions in the model,
* N: number of reactions in a subsystem,
* M: number of reactions in the model.

$P(x \geq k) = 1 - hypergeom.cdf(k-1, M, n, N)$


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.decomposition import PCA
from itertools import permutations, product, combinations
from scipy.stats import pearsonr, spearmanr, mannwhitneyu, hypergeom

from itertools import permutations
from itertools import combinations

In [2]:
#https://www.scribbr.com/statistics/two-way-anova/
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multitest as multi

In [3]:
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning, HessianInversionWarning, ValueWarning
# ignore these warning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=HessianInversionWarning)
warnings.filterwarnings("ignore", category=ValueWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

## Settings

In [19]:
analysis = "Fastcore"
#analysis = "iMAT"

#analysis = "gimme"
#analysis = "init"
#analysis = "tinit"

In [20]:
analysis_type = "FVA"
#analysis_type = "pFBA"

In [21]:
fdr = True

## Read the data

In [22]:
reactions =  pd.read_csv("data\\" + analysis_type + "_" + analysis + ".csv", sep=";").iloc[:,0]

In [23]:
if fdr:
    df_genotype = pd.read_csv("results_pairs\\" + analysis_type + "_"+ analysis + "_genotype_compare_q.csv")
    df_gender = pd.read_csv("results_pairs\\" + analysis_type + "_"+ analysis + "_gender_compare_q.csv")
    df_diet = pd.read_csv("results_pairs\\" + analysis_type + "_"+ analysis + "_diet_compare_q.csv")
else:
    df_genotype = pd.read_csv("results_pairs\\" + analysis_type + "_" + analysis + "_genotype_compare.csv")
    df_gender = pd.read_csv("results_pairs\\" + analysis_type + "_" + analysis + "_gender_compare.csv")
    df_diet = pd.read_csv("results_pairs\\" + analysis_type + "_" + analysis + "_diet_compare.csv")

In [24]:
tests_genotype = list(df_genotype.columns[1:])
tests_gender = list(df_gender.columns[1:])
tests_diet = list(df_diet.columns[1:])

### Fill the analysis data with all the reactions
As a basis I take the union of the reactions included in the selected group of models.

In [25]:
df_reactions = pd.DataFrame(columns=["rxn"])
df_reactions["rxn"] = reactions

df_genotype = pd.merge(df_genotype, df_reactions, how="outer").fillna(1)
df_gender = pd.merge(df_gender, df_reactions, how="outer").fillna(1)
df_diet = pd.merge(df_diet, df_reactions, how="outer").fillna(1)

### Get the subsystems data

In [26]:
df_subsystems = pd.read_csv("models\\iMM865_subsystems.txt", sep=";")
df_subsystems_f = df_subsystems.copy()
df_subsystems_f['rxn'] = df_subsystems_f['rxn']+'_f'
df_subsystems_b = df_subsystems.copy()
df_subsystems_b['rxn'] = df_subsystems_b['rxn']+'_b'

df_subsystems = pd.concat((df_subsystems, df_subsystems_b, df_subsystems_f), ignore_index=True).reindex()

df_subsystems.head()

,rxn,subsystem
0,10FTHF5GLUtl,"Transport, lysosomal"
1,10FTHF5GLUtm,"Transport, mitochondrial"
2,10FTHF6GLUtl,"Transport, lysosomal"
3,10FTHF6GLUtm,"Transport, mitochondrial"
4,10FTHF7GLUtl,"Transport, lysosomal"


Keep only the reactions that are present in the observed models

In [27]:
df_subsystems = df_subsystems[df_subsystems.rxn.isin(reactions)]

In [28]:
subsystems = df_subsystems.subsystem.dropna().unique()

### Merge

In [29]:
df_genotype = pd.merge(df_genotype, df_subsystems, how="left")
df_gender = pd.merge(df_gender, df_subsystems, how="left")
df_diet = pd.merge(df_diet, df_subsystems, how="left")

In [30]:
df_genotype = df_genotype[['rxn', 'subsystem'] + tests_genotype]
df_gender = df_gender[['rxn', 'subsystem'] + tests_gender]
df_diet = df_diet[['rxn', 'subsystem'] + tests_diet]

## Analysis

In [31]:
"""
GENOTYPE
"""
df_genotype_enrich = pd.DataFrame(columns = ['subsystem'] + tests_genotype)
df_genotype_enrich['subsystem'] = subsystems

n_all = len(reactions)

for test in tests_genotype:
    df_test = df_genotype[[test,'subsystem']]
    n_signif_all = (df_test[test] < 0.05).sum()         
    for subsystem in subsystems:
        df_sub = df_test[df_test.subsystem == subsystem]
        n_sub = len(df_sub)
        n_signif_sub = (df_sub[test] < 0.05).sum()
        
        M = n_all # all reactions in a model
        n = n_signif_all # all significant 
        N = n_sub # reactions in a subsystem
        k = n_signif_sub # significant in a subsystem
        
        if n:         
            p = 1 - hypergeom.cdf(k-1, M, n, N)                
        else:
            p = 1.0
            
        df_genotype_enrich.loc[(df_genotype_enrich['subsystem'] == subsystem), test] = p
        
        #print(k, M, n, N)     

KeyboardInterrupt: 

In [ ]:
"""
GENDER
"""
df_gender_enrich = pd.DataFrame(columns = ['subsystem'] + tests_gender)
df_gender_enrich['subsystem'] = subsystems

n_all = len(reactions)

for test in tests_gender:
    df_test = df_gender[[test,'subsystem']]
    n_signif_all = (df_test[test] < 0.05).sum()         
    for subsystem in subsystems:
        df_sub = df_test[df_test.subsystem == subsystem]
        n_sub = len(df_sub)
        n_signif_sub = (df_sub[test] < 0.05).sum()
        
        M = n_all # all reactions in a model
        n = n_signif_all # all significant 
        N = n_sub # reactions in a subsystem
        k = n_signif_sub # significant in a subsystem
        
        if n:         
            p = 1 - hypergeom.cdf(k-1, M, n, N)                
        else:
            p = 1.0
            
        df_gender_enrich.loc[(df_gender_enrich['subsystem'] == subsystem), test] = p
        
        #print(k, M, n, N)     

In [32]:
"""
DIET
"""
df_diet_enrich = pd.DataFrame(columns = ['subsystem'] + tests_diet)
df_diet_enrich['subsystem'] = subsystems

n_all = len(reactions)


for test in tests_diet:
    df_test = df_diet[[test,'subsystem']]
    n_signif_all = (df_test[test] < 0.05).sum()         
    
    
    for subsystem in subsystems:
        df_sub = df_test[df_test.subsystem == subsystem]
        n_sub = len(df_sub)
        n_signif_sub = (df_sub[test] < 0.05).sum()
                
        
        k = n_signif_sub # significant in a subsystem
        M = n_all # all reactions in a model
        n = n_signif_all # all significant 
        N = n_sub # reactions in a subsystem
        
        
        if n:         
            p = 1 - hypergeom.cdf(k-1, M, n, N)                
        else:
            p = 1.0

        df_diet_enrich.loc[(df_diet_enrich['subsystem'] == subsystem), test] = p
        
        if subsystem.startswith("Chole") and test.startswith("(HFnC"):
            print(test)
            print("k, M, n, N:", k, M, n, N)   
            print("p",p)
        

(HFnC:HFC)
k, M, n, N: 73 6781 140 241
p 0.0
(HFnC<HFC)
k, M, n, N: 4 6781 8 241
p 9.73414704429354e-05
(HFnC>HFC)
k, M, n, N: 63 6781 87 241
p 4.728439861878542e-13


In [19]:
# at least 0 reactions are changed
1 - hypergeom.cdf(-1, 6781, 110, 10)

1.0

In [29]:
1 - hypergeom.cdf(2, 6781, 110, 10)

0.00045877694824025816

## Save the results

In [20]:
df_genotype_enrich.to_csv("results_enrich\\" + analysis_type + "_" + analysis + "_pairs_genotype_enrich.csv", index=False)
df_gender_enrich.to_csv("results_enrich\\" + analysis_type + "_" + analysis + "_pairs_gender_enrich.csv", index=False)
df_diet_enrich.to_csv("results_enrich\\" + analysis_type + "_" + analysis + "_pairs_diet_enrich.csv", index=False)

In [21]:
df_genotype_enrich_q = df_genotype_enrich.copy()
df_gender_enrich_q = df_gender_enrich.copy()
df_diet_enrich_q = df_diet_enrich.copy()

for c in df_genotype_enrich_q.columns[1:]:    
    df_genotype_enrich_q[c] = multi.multipletests(df_genotype_enrich_q[c], method = 'fdr_bh')[1]
for c in df_gender_enrich_q.columns[1:]:
    df_gender_enrich_q[c] = multi.multipletests(df_gender_enrich_q[c], method = 'fdr_bh')[1]
for c in df_diet_enrich_q.columns[1:]:
    df_diet_enrich_q[c] = multi.multipletests(df_diet_enrich_q[c], method = 'fdr_bh')[1]

In [22]:
df_genotype_enrich_q.to_csv("results_enrich\\" + analysis_type + "_" + analysis + "_pairs_genotype_enrich_q.csv", index=False)
df_gender_enrich_q.to_csv("results_enrich\\" + analysis_type + "_" + analysis + "_pairs_gender_enrich_q.csv", index=False)
df_diet_enrich_q.to_csv("results_enrich\\" + analysis_type + "_" + analysis + "_pairs_diet_enrich_q.csv", index=False)

In [23]:
df_diet_enrich_q.loc[df_diet_enrich_q.subsystem.str.startswith("Sq"), ["subsystem","M_WT_(HFnC>HFC)", "M_WT_(HFnC<HFC)", "M_WT_(HFnC:HFC)"]]


,subsystem,M_WT_(HFnC>HFC),M_WT_(HFnC<HFC),M_WT_(HFnC:HFC)


In [24]:
df_genotype_enrich_q[(df_genotype_enrich_q[df_genotype_enrich_q.columns[1:]] < 0.05).any(axis=1)]

,subsystem,F_LFnC_(WT:KO),F_LFnC_(WT<KO),F_LFnC_(WT>KO),F_HFnC_(WT:KO),F_HFnC_(WT<KO),F_HFnC_(WT>KO),F_HFC_(WT:KO),F_HFC_(WT<KO),F_HFC_(WT>KO),...,LFnC_(WT>KO),HFnC_(WT:KO),HFnC_(WT<KO),HFnC_(WT>KO),HFC_(WT:KO),HFC_(WT<KO),HFC_(WT>KO),(WT:KO),(WT<KO),(WT>KO)


In [25]:
df_gender_enrich_q[(df_gender_enrich_q[df_gender_enrich_q.columns[1:]] < 0.05).any(axis=1)]

,subsystem,WT_LFnC_(F:M),WT_LFnC_(F<M),WT_LFnC_(F>M),WT_HFnC_(F:M),WT_HFnC_(F<M),WT_HFnC_(F>M),WT_HFC_(F:M),WT_HFC_(F<M),WT_HFC_(F>M),...,LFnC_(F>M),HFnC_(F:M),HFnC_(F<M),HFnC_(F>M),HFC_(F:M),HFC_(F<M),HFC_(F>M),(F:M),(F<M),(F>M)
23,Tyrosine metabolism,1,0.00102963,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
45,Sphingolipid metabolism,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [26]:
df_diet_enrich_q[(df_diet_enrich_q[df_diet_enrich_q.columns[1:]] < 0.05).any(axis=1)]

,subsystem,F_WT_(LFnC:HFnC),F_WT_(LFnC<HFnC),F_WT_(LFnC>HFnC),F_WT_(LFnC:HFC),F_WT_(LFnC<HFC),F_WT_(LFnC>HFC),F_WT_(HFnC:HFC),F_WT_(HFnC<HFC),F_WT_(HFnC>HFC),...,KO_(HFnC>HFC),(LFnC:HFnC),(LFnC<HFnC),(LFnC>HFnC),(LFnC:HFC),(LFnC<HFC),(LFnC>HFC),(HFnC:HFC),(HFnC<HFC),(HFnC>HFC)
3,"Glycine, serine, alanine, and threonine metabo...",1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
12,"Transport, peroxisomal",1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
